In [2]:
%matplotlib inline
import pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
help(camhd)
help(requests)

In [ ]:
import requests
import csv
scene_log_url = 'https://raw.githubusercontent.com/ooi-data-review/camhd_metadata/master/scene_timing/zoom0_scenes.csv'
scene_log_page = requests.get(scene_log_url)
scene_log_content = scene_log_page.content.decode('utf-8')
scene_log_csv = csv.reader(scene_log_content.splitlines(), delimiter=',')
scene_log = list(scene_log_csv)

In [ ]:
import glob
filenames = glob.glob('/data/*.mov')

In [ ]:
frame_numbers = []
for filename in filenames:
    for row in scene_log:
        if filename[6:32] in row[0]:
            file_exists = True;
            next_frame_time = row[9].split(':') # this is the seventh scene showing a bacterial mat at the base of Mushroom
            if len(next_frame_time)==3:
                frame_time = next_frame_time
    if file_exists:
        frame_numbers.append(int(round((int(frame_time[1])*60+int(frame_time[2]))*29.95))+60)
    else:
        frame_numbers.append(99999)
    file_exists = False

In [ ]:
plt.rc('figure', figsize=(8, 8))
frame = camhd.get_frame(filenames[0], frame_numbers[0], 'rgb24')
imgplot = plt.imshow(frame)

In [ ]:
from numpngw import write_png
from scipy.misc import imresize
write_png('time_lapse.png', imresize(frame, (270, 480)))

In [ ]:
%%time
import subprocess as sp
command = [ 'ffmpeg',
           '-y', #overwrite output file if it exists
           '-f', 'rawvideo',
           '-vcodec','rawvideo',
           '-s', '1920x1080', # size of input frame
           '-pix_fmt', 'rgb24',
           '-r', '30', # output frame rate
           '-i', '-', # input from pipe
           '-an', # no audio
           '-vf', 'scale=480x270',
           '-c:v', 'h264',
           '-preset', 'veryfast',
           '-crf', '18',
           '-pix_fmt', 'yuv420p',
           'time_lapse.mp4' ]

pipe = sp.Popen(command, stdin=sp.PIPE, stderr=sp.PIPE)

for i, filename in enumerate(filenames):
    if frame_numbers[i] != 99999:
        frame = camhd.get_frame(filename, frame_numbers[i], 'rgb24')
        pipe.stdin.write(frame.tostring())

pipe.stdin.flush() # Ensure nothing is left in the buffer
pipe.terminate()

In [ ]:
%%HTML
<video width="480" height="270" controls poster="test.png">
  <source src="time_lapse.mp4" type="video/mp4">
</video>